In [2]:
#pip install tweepy==3.8.0
#!pip install pylast

In [3]:
# Imports
import tweepy
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy.streaming import StreamListener

import numpy as np
import pandas as pd
import csv

import json
import sys
sys.setrecursionlimit(1500)

from IPython.core.display import HTML
import pylast

In [4]:
# Build credentials dictionary
credentials = {}

credentials['consumer_key'] = '2Zz9e25p58LJsLJoVYg9hZQ6s'
credentials['consumer_secret'] = 'uEYmV56gDwhb8wVmIV72NZ2DrcRXCuRm1N9mSSN8DanwopzAB9'
credentials['access_token'] = '1190404328646791168-9wEBKsywoKg3ZC4wksEnM0siJzf3ZT'
credentials['access_secret'] = 'u5QdXAG5JTjAm122zqDyF0wL7dq3qcI0ZExqbcALqVmoX'

# Write credentials to file
with open("twitter_credentials.json", "w") as file:
    json.dump(credentials, file)

# Read credentials from file
twit_cred_file = open('twitter_credentials.json', 'r')
twitter_credentials = json.loads(twit_cred_file.read())

In [5]:
auth = OAuthHandler(twitter_credentials['consumer_key'], twitter_credentials['consumer_secret'])
auth.set_access_token(twitter_credentials['access_token'], twitter_credentials['access_secret'])
 
api = tweepy.API(auth)

In [6]:
df = pd.DataFrame({})

# Define a class that will pull tweets
class StreamListener(tweepy.StreamListener):
    
    # Method defined in order to append tweets to a Pandas DataFrame
    def on_status(self, status):
        
        # If location data was enabled by the user
        if status.user.location is not None:
            
            # Allow dataframe changes outside the scope of this function
            global df
            
            # Append text and location data to a row in the dictionary without the index
            df = df.append({'text': status.text, 'location': status.user.location}, ignore_index=True)
            
            # Print the tweet text, status is the name ascribed to a Tweet
            print(status.text)
            
    # Define a method that will catch when too many tweets have been requested of Twitter
    # "Enhance Your Calm" error
    def on_error(self, status_code):
        
        # Check to see if the status code is equal to 420
        if status_code == 420:
            
            # Returns false if this is the case
            return False

In [7]:
# Instantiate the StreamListener class
disaster_listener = StreamListener()

# Create a stream object using Tweepy's stream method
stream = tweepy.Stream(auth=api.auth, listener=disaster_listener)

# Filter Tweepy's stream object by Tweets about wildfires in the English language
# This cell was only ran once to gather Tweets in real time and does not need to be ran again
#stream.filter(track=['#wildfires'], languages=['en'])

In [8]:
# This cell had a meaningful output during data collection
df.head()

""


In [ ]:
# #Inputting the credentials here
# consumer_key = "2Zz9e25p58LJsLJoVYg9hZQ6s"
# consumer_secret = "uEYmV56gDwhb8wVmIV72NZ2DrcRXCuRm1N9mSSN8DanwopzAB9"
# access_token = "1190404328646791168-9wEBKsywoKg3ZC4wksEnM0siJzf3ZT"
# access_secret = "u5QdXAG5JTjAm122zqDyF0wL7dq3qcI0ZExqbcALqVmoX"

# auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# auth.set_access_token(access_token, access_secret)

# api = tweepy.API(auth, wait_on_rate_limit=True)

# # Open/create a file to append data to
# csvFile = open('result.csv', 'a')

# #Use csv writer
# csvWriter = csv.writer(csvFile)
    
# for tweet in tweepy.Cursor(api.search, count=100,
#                            geocode="18.2208,66.5901,0.5mi",
#                            lang="en",
#                            since="2017-09-16",
#                            until="2017-10-02").items():
#     print (tweet.created_at, tweet.text)
#     csvWriter.writerow([tweet.created_at, tweet.text.encode('utf-8')])

In [40]:
# tweet = api.get_status(id='910523398253461000')
# print(tweet.text)

In [8]:
tweets = [{}]
for tweet in tweepy.Cursor(api.search, q="#wildfires", count=100, lang="en", since="2017-09-16").items():
                          
    tweets.append({'Created At': tweet.created_at, 'Message': tweet.text})

TweepError: Failed to send request: ('Connection broken: OSError("(10054, \'WSAECONNRESET\')")', OSError("(10054, 'WSAECONNRESET')"))

In [9]:
df = pd.DataFrame(tweets)

In [11]:
df.shape

(6261, 2)

In [12]:
df.rename({'Created At': 'date_created', 'Message': 'message'}, axis=1, inplace=True)
df.dropna(inplace=True)

In [13]:
df

,date_created,message
1,2019-11-06 04:00:21,California #artist Danielle Nelisse paints #ab...
2,2019-11-06 03:58:48,Many low income people lost their homes in Baj...
3,2019-11-06 03:58:37,RT @EpochTimes: “Get your act together Governo...
4,2019-11-06 03:57:50,RT @EpochTimes: “Get your act together Governo...
5,2019-11-06 03:54:22,Yikes\nhttps://t.co/gs4D0VP4HH #wildfires #Africa
...,...,...
6256,2019-10-31 03:36:52,RT @ChaliceGarden: URGENT! Have a horse traile...
6257,2019-10-31 03:36:35,RT @ladailynews: @sullikevphoto Here’s where y...
6258,2019-10-31 03:36:17,RT @AHMalcolm: California #wildfires near the ...
6259,2019-10-31 03:36:00,California #wildfires near the Reagan Presiden...


In [16]:
df.to_csv('./data/firetweets.csv', index=False)